In [1]:
using DelimitedFiles, PyPlot, Statistics, LinearAlgebra


tomo_data=readdlm("tomo_data.txt"); # data set
xs=tomo_data[:,1]; # x coordinates of sources
zs=tomo_data[:,2]; # z coordinates of sources
xr=tomo_data[:,3]; # x coordinates of receiver
zr=tomo_data[:,4]; # z coordinates of receiver
t=tomo_data[:,5]; # Travel times;


In [ ]:
Δzs=zeros(Float64,length(zs));

for i=1:length(zs)-1
    
    if zs[i+1] == zs[i];
        
        Δzs[i] = 0.0;
    
    else
        Δzs[i]= zs[i+1] - zs[i]
    
    end
end


Δ̄zₛ = maximum(Δzs)



Δzr=zeros(Float64,length(zr));

for i=1:length(zr)-1
    
    if zr[i+1] == zr[i];
        
        Δzr[i] = 0.0;
    
    else
        Δzr[i]= zr[i+1] - zr[i]
    
    end
end


Δ̄zᵣ = mean(Δzr)
    

Δ̄zs = 8.130
ns = Int(floor(maximum(zs)/ Δ̄zs))+ 1;
Δ̄zr=8.122; 
nr = Int(floor(maximum(zr)/ Δ̄zr))+ 1;
ns

In [2]:
#Show how to compute sources adn receivers:

ns=50;
nr=50;
#Grid points:
grid=(100,100)


#Vectors for sources and receivers!

sx=zeros(Float64,ns);
sz=LinRange(minimum(zs),maximum(zs),ns);

sources=[sx sz];

rx=300.0*ones(Float64,ns);
rz=LinRange(minimum(zr),maximum(zr),nr);

sources=[sx sz]; receivers=[rx rz];



#Call the functio nto build the forward model


In [ ]:
maximum(sz)

In [9]:
 
gx=100; gz=100; scrd=0; rcrd=300; ymax=400

nel=50; 
xrwell=fill(rcrd,nel); xswell=fill(scrd,nel);
yrwell=LinRange(1.25,399.25,nel); yswell=LinRange(0.6,399,nel);
rwell=[xrwell yrwell]; swell=[xswell yswell];            #OJO changing data type
L=zeros(length(xrwell)*length(xswell),gx*gz);Js=Float64[];cellnum=Int[];Is=Float64[];Vs=Float64[]; rayn=0;spV=Int[];
#Grid definition

xgrid=LinRange(0,maximum(xr),gx+1);
dx=xgrid[2]-xgrid[1]
xgrid=collect(xgrid[2:end-1])
ygrid=LinRange(0,maximum(zr),gz+1);
dz=ygrid[2]-ygrid[1]
ygrid=collect(ygrid[2:end-1])


    for s in 1:length(swell[:,1])
        for r in 1:length(rwell[:,1])
                p1=[rwell[r,2] swell[s,2]];
                #println("p1 ",p1) 
                p2=[rwell[r,1] swell[s,1]];
                #println("p2 ",p2)
                m=(rwell[r,2]-swell[s,2])/(rwell[r,1]-swell[s,1]); #Slope of the ray
                b=rwell[r,2]-(rwell[r,1]*m); #Intercept of the ray
                #println(m," ",b)
                #Determination grid points involved in distance calculation
                xcross=((ygrid.-b)./m); # Variable X at the fixed Y of the grid OJO Round
                ycross=((m.*xgrid).+b); # Variable Y at the fixed X of the grid OJO Round
                #println(xcross)
                #println(ycross)
                t1=[ycross' ygrid' p1;  xgrid' xcross' p2];
                #println(t1)
                t2=unique(sortslices(round.(t1;digits=2),dims=2,by=x->x[2],rev=false),dims=2); #Sorted array with all the points
                #println("T2 ", t2)
                #x=0;
                #tt=(findall(x->x>rcrd||x<scrd||isequal(x,NaN),t2));
                tt1= findall(x->x>rcrd||x<scrd,t2[2,:])
                tt2= findall(x->x>ymax||x<scrd,t2[1,:])
                tt=[tt1;tt2]
                #println(tt)
                #println(typeof(tt))
                #tt=vcat(tt1,tt2)
                #println("Index to delete :",tt1,tt2)
                a=zeros(length(tt))
                for i in 1:length(tt)
                    a[i]=tt[i][1]; #loop to extract the indices of the collumns that are out of bounds
                end
                crossmat =t2[:, 1:end .∉[a]]; #Final Cross matrix without undesired value
 #println("crossmat ",crossmat)
                #distance calculator
                d=zeros(length(crossmat[1,:])-1);
                for i in 1:length(crossmat[1,:])-1
                    d[i]=sqrt(((crossmat[1,i+1]-crossmat[1,i])^2)+(crossmat[2,i+1]-crossmat[2,i])^2);
                end
                #println("Distances :", d)
                td=sum(d);
                #Cell definition
                    c=zeros(2,length(crossmat[1,:])-1);
                for i in 1:2
                    for j in 1:length(crossmat[1,:])-1
                            c[i,j]=((crossmat[i,j+1]-crossmat[i,j])/2)+crossmat[i,j] #Calculation of segment center
                    end
                end
                #println("Centers of segments", c)            
                cellnum1=zeros(length(d))
                for i in 1:length(d)
                    cellnum1[i]=ceil(c[1,i]/dz)+(((ceil(c[2,i]/dx))-1)*gz)#+(gz*(temp3[2,i]-1)); #Calculation of cell number
                end

                rayn= rayn+1

                for i in 1:length(cellnum1)
                    L[rayn,Int(cellnum1[i])]=d[i]
                end
        
    end
end




In [10]:
L

2500×10000 Matrix{Float64}:
 3.00002  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0      0.0  0.0      0.0  0.0
 3.00135  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.00481  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.0104   0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.0181   0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.02789  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0      0.0  0.0      0.0  0.0
 3.03975  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.05555  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.07174  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.08992  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.11005  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0      0.0  0.0      0.0  0.0
 3.13209  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.15601  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0

In [6]:
#Show how to compute sources adn receivers:

ns=50;
nr=50;
#Grid points:
grid=(100,100)


#Vectors for sources and receivers!

sx=zeros(Float64,ns);
sz=LinRange(minimum(zs),maximum(zs),ns);

rx=300.0*ones(Float64,ns);
rz=LinRange(minimum(zr),maximum(zr),nr);

sources=[sx sz]; receivers=[rx rz];


#Call the functio nto build the forward model

In [ ]:
sources == 

In [7]:
function DistanceMatrix(sources, receivers, grid)
    
    #Define grid:
    
    gx,gz=grid

    xgrid=LinRange(0.0,300,gx+1);
    dx=xgrid[2]-xgrid[1];
    xgrid=collect(xgrid[2:end-1]);
    zgrid=LinRange(0.0,400,gz+1);
    dz=zgrid[2]-zgrid[1];
    zgrid=collect(zgrid[2:end-1]);
        
    #Allocation variables
    D=zeros(Float64,(length(sources[:,1])*length(receivers[:,1]),gx*gz));
    ray_number= 0; 
    
    
     for s in 1:length(sources[:,1])
        for r in 1:length(receivers[:,1])
        
            point1=[receivers[r,2] sources[s,2]];
            point2=[receivers[r,1] sources[s,1]];
                
            #Compute m and b in a function get ray param
            m=(receivers[r,2]-sources[s,2])/(receivers[r,1]-sources[s,1]); #Slope of the ray
            b=receivers[r,2]-(receivers[r,1]*m); #Intercept of the ray
        
            #Compute grid points involved in calculations, Put into a matrix
            
            cx=((zgrid.-b)./m); 
            cz=((m.*xgrid).+b); 
               
            t1=[cz' zgrid' point1;  xgrid' cx' point2];
            CP=unique(sortslices(round.(t1;digits=2),dims=2),dims=2);

            indx1= findall(x->  x > 300.0 || x < 0.0 ,CP[2,:]) #rplace with variables
            indx2= findall(x->  x > 400.0 || x < 0.0 ,CP[1,:]) #rplace with variables
            
            indx=[indx1; indx2]

            aux=zeros(length(indx))
            
            for i in 1:length(indx)
                aux[i]=indx[i][1]; #loop to extract the indices of the collumns that are out of bounds
            end
                
            CP =CP[:, 1:end .∉[aux]]; #Final Cross matrix without undesired value
            
            #compute the distances
        
            dist=zeros(length(CP[1,:])-1);

            for i in 1:length(CP[1,:])-1
                dist[i]=sqrt(((CP[1,i+1]-CP[1,i])^2)+(CP[2,i+1]-CP[2,i])^2);
            end

            #Compute the centers;
        
            centers=zeros(2,length(CP[1,:])-1);

            for i in 1:2
                for j in 1:length(CP[1,:])-1
                    centers[i,j]=((CP[i,j+1]-CP[i,j])/2)+CP[i,j] #Calculation of segment center
                end
            end
        
            cell_id=zeros(length(dist))
     
            for i in 1:length(dist)
                cell_id[i]=ceil(centers[1,i]/dz)+(((ceil(centers[2,i]/dx))-1)*gz)
            end

    
            ray_number= ray_number+1;

            for j=1:length(cell_id)
                D[ray_number,Int(cell_id[j])] = dist[j]
                end
        end
    
    end

    return D
    
end

        
        

DistanceMatrix (generic function with 1 method)

In [ ]:
σ=0.0004

In [8]:
D= DistanceMatrix(sources, receivers, grid)

2500×10000 Matrix{Float64}:
 3.00002  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0      0.0  0.0      0.0  0.0
 3.00135  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.00481  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.0104   0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.0181   0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.02789  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0      0.0  0.0      0.0  0.0
 3.03975  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.05555  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.07174  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.08992  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.11005  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0      0.0  0.0      0.0  0.0
 3.13209  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0.0
 3.15601  0.0  0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0      0.0  0

In [11]:
 L == D

false

In [ ]:
function L_build(rwell,swell,gx,gz,scrd,rcrd,xgrid,ygrid,ymax,dx,dz)
    L=zeros(length(xrwell)*length(xswell),gx*gz);Js=Float64[];cellnum=Int[];Is=Float64[];Vs=Float64[]
    rayn=0;spV=Int[];
    for s in 1:length(swell[:,1])
        for r in 1:length(rwell[:,1])
                p1=[rwell[r,2] swell[s,2]];
                #println("p1 ",p1) 
                p2=[rwell[r,1] swell[s,1]];
                #println("p2 ",p2)
                m=(rwell[r,2]-swell[s,2])/(rwell[r,1]-swell[s,1]); #Slope of the ray
                b=rwell[r,2]-(rwell[r,1]*m); #Intercept of the ray
                #println(m," ",b)
                #Determination grid points involved in distance calculation
                xcross=((ygrid.-b)./m); # Variable X at the fixed Y of the grid OJO Round
                ycross=((m.*xgrid).+b); # Variable Y at the fixed X of the grid OJO Round
                #println(xcross)
                #println(ycross)
                t1=[ycross' ygrid' p1;  xgrid' xcross' p2];
                #println(t1)
                t2=unique(sortslices(round.(t1;digits=2),dims=2,by=x->x[2],rev=false),dims=2); #Sorted array with all the points
                #println("T2 ", t2)
                #x=0;
                #tt=(findall(x->x>rcrd||x<scrd||isequal(x,NaN),t2));
                tt1= findall(x->x>rcrd||x<scrd,t2[2,:])
                tt2= findall(x->x>ymax||x<scrd,t2[1,:])
                tt=[tt1;tt2]
                #println(tt)
                #println(typeof(tt))
                #tt=vcat(tt1,tt2)
                #println("Index to delete :",tt1,tt2)
                a=zeros(length(tt))
                for i in 1:length(tt)
                    a[i]=tt[i][1]; #loop to extract the indices of the collumns that are out of bounds
                end
                crossmat =t2[:, 1:end .∉[a]]; #Final Cross matrix without undesired value
                #println("crossmat ",crossmat)
                #distance calculator
                d=zeros(length(crossmat[1,:])-1);
                for i in 1:length(crossmat[1,:])-1
                    d[i]=sqrt(((crossmat[1,i+1]-crossmat[1,i])^2)+(crossmat[2,i+1]-crossmat[2,i])^2);
                end
                #println("Distances :", d)
                td=sum(d);
                #Cell definition
                    c=zeros(2,length(crossmat[1,:])-1);
                for i in 1:2
                    for j in 1:length(crossmat[1,:])-1
                            c[i,j]=((crossmat[i,j+1]-crossmat[i,j])/2)+crossmat[i,j] #Calculation of segment center
                    end
                end
                #println("Centers of segments", c)            
                cellnum=zeros(length(d))
                for i in 1:length(d)
                    cellnum[i]=ceil(c[1,i]/dz)+(((ceil(c[2,i]/dx))-1)*gz)#+(gz*(temp3[2,i]-1)); #Calculation of cell number
                end

In [ ]:

m, b = GetRayParam(zr,xr,zs,xs);

#lx=101;
gx= LinRange(0.0,300.0,lx);
dx= gx[3] - gx[2]

#lz=101;
gz=LinRange(0.0,400.0,lz);
dz= gz[3] - gz[2]

nr=length(t)


D=zeros(Float64,(nr,(length(gz)-1)*(length(gx)-1)))
z=zeros(length(gx)); x=zeros(length(gz))






# Get the intersection for each ray with x and z.
for r=1:nr
    
    for i=1:length(gx)
        z[i] = m[r]*gx[i] .+ b[r] #ray    
    end

    for j=1:length(gz) 
        x[j]= (gz[j] - b[r])/m[r] ;
    end


# Organize the point pairs into two vectors. 

    Iz= [gx z]; 
    Ix= [x gz];

    CP=[Iz; Ix]; 
    CP = sortslices(CP, dims=1)

    indx1= findall(x->  x > 300.0 || x < 0.0 ,CP[:,1])
    indx2= findall(x->  x > 400.0 || x < 0.0 ,CP[:,2])
    CP= CP[1:end .∉[indx1], 1:end]; 
    CP= CP[1:end,1:end .∉[indx2]]; 
#I= I[1:end, 1:end .∉[indx2]]; 


    dist=zeros(size(CP,1)-1);
    

    for i=1:size(CP,1)-1;
        dist[i]=  d(CP[i,1],CP[i+1,1],CP[i,2],CP[i+1,2]); 
    end


    dist=round.(dist, digits=2)
#Cell definition
    c=zeros(size(CP,1)-1,2);
                

    for j in 1:size(CP,2)
        for i in 1:size(CP,1)-1
            c[i,j]=((CP[i+1,j]-CP[i,j])/2)+CP[i,j] #Calculation of segment center
        end
    end

    c=round.(c,digits=2)
    
    
    id=zeros(length(dist))

    nz=length(gz)-1 #Number of cels in the grid that I defined.


    for i in 1:length(dist)
        id[i]= ceil(c[i,2]/dz)+(((ceil(c[i,1]/dx))-1)*nz) #(ceil(c[i,1]/dz))+(((ceil(c[i,2]/dx))-1)*gz)
    end


    for j=1:length(id)
        index=Int(id[j])
        D[r,index] = dist[j]
    end


end
